In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from functools import reduce
import unicodedata
from pyspark.sql.functions import col

In [37]:
#iniciar spark

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [38]:
df = spark.read.option('delimiter', ';').option('header', 'true').option("encoding","ISO-8859-1").csv('s3a://catarina-3sprint-sptech-bucket-bruto/controle_mensal_parametros_basicos_2022.csv')

In [39]:
df.show()


+-----------------+---+-----------------+-----------+-------------+-------------------+--------------------+--------------------+-------------------+---------------------------------+---------------------------------+------------------------------+-----------------------------+------------------------------+--------------------+-----------------+-----------------+-----------------+----------------------+--------------------+--------------------+-----+
|Região Geográfica| UF|Regional de Saúde|Código IBGE|    Município|Tipo da Instituição|Sigla da Instituição| Nome da Instituição|CNPJ da Instituição|Nome do escritório regional/local|CNPJ do escritório regional/local|Tipo da Forma de Abastecimento|Código Forma de abastecimento|Nome da Forma de Abastecimento|   Nome da ETA / UTA|Tipo de Filtração|Ano de referência|Mês de referência|Ponto de Monitoramento|           Parâmetro|               Campo|Valor|
+-----------------+---+-----------------+-----------+-------------+-------------------+-

In [40]:
# Limpando a tabela
demaisParametros=df.drop('Código IBGE',' Município','Sigla da Instituição','Código Forma de abastecimento',
                  'CNPJ da Instituição','Nome do escritório regional/local','CNPJ do escritório regional/local',
                  ' Nome da ETA / UTA','Regional de Saúde''ipo da Forma de Abastecimento','Nome da Forma de Abastecimento',
                  'Nome da ETA','Regional de Saúde','Nome da ETA / UTA','Tipo da Forma de Abastecimento','Tipo da Instituição',
                  'Nome da Instituição','Município')
demaisParametros.show()

+-----------------+---+-----------------+-----------------+-----------------+----------------------+--------------------+--------------------+-----+
|Região Geográfica| UF|Tipo de Filtração|Ano de referência|Mês de referência|Ponto de Monitoramento|           Parâmetro|               Campo|Valor|
+-----------------+---+-----------------+-----------------+-----------------+----------------------+--------------------+--------------------+-----+
|          SUDESTE| MG|    SEM FILTRAÇÃO|             2022|                3|  Pós-filtração ou ...|       Turbidez (uT)|Número de amostra...|    8|
|          SUDESTE| MG|    SEM FILTRAÇÃO|             2022|                3|  Pós-filtração ou ...|       Turbidez (uT)|        Percentil 95| 0,29|
|          SUDESTE| MG|    SEM FILTRAÇÃO|             2022|                3|  Pós-filtração ou ...|       Turbidez (uT)|Número de dados >...|    0|
|          SUDESTE| MG|    SEM FILTRAÇÃO|             2022|                3|  Pós-filtração ou ...|      

In [41]:
demaisParametros.withColumnRenamed("Região Geográfica","Regiao").withColumnRenamed("Mês de referência","mes").withColumnRenamed("Ano de referência","ano").show()


+-------+---+-----------------+----+---+----------------------+--------------------+--------------------+-----+
| Regiao| UF|Tipo de Filtração| ano|mes|Ponto de Monitoramento|           Parâmetro|               Campo|Valor|
+-------+---+-----------------+----+---+----------------------+--------------------+--------------------+-----+
|SUDESTE| MG|    SEM FILTRAÇÃO|2022|  3|  Pós-filtração ou ...|       Turbidez (uT)|Número de amostra...|    8|
|SUDESTE| MG|    SEM FILTRAÇÃO|2022|  3|  Pós-filtração ou ...|       Turbidez (uT)|        Percentil 95| 0,29|
|SUDESTE| MG|    SEM FILTRAÇÃO|2022|  3|  Pós-filtração ou ...|       Turbidez (uT)|Número de dados >...|    0|
|SUDESTE| MG|    SEM FILTRAÇÃO|2022|  3|  Pós-filtração ou ...|       Turbidez (uT)|Número de dados >...|    0|
|SUDESTE| MG|    SEM FILTRAÇÃO|2022|  3|  Pós-filtração ou ...|       Turbidez (uT)|Número de dados >...|    0|
|SUDESTE| MG|    SEM FILTRAÇÃO|2022|  3|  Pós-filtração ou ...|       Turbidez (uT)|Número de dados <...

In [42]:
# UPANDO OS DADOS TRATADOS COM O 1º TRATAMENTO

#demaisParametros.write.mode('overwrite').csv('s3a://catarina-3sprint-sptech-bucket-tratados/demais-parametros')


In [43]:
## 2ª ETAPA DO TRATAMENTO

In [44]:
dfTurbidez = demaisParametros.select('Região Geográfica','Parâmetro','valor').where(demaisParametros['Parâmetro'] == 'Turbidez (uT)')

In [45]:
dfTurbidez.show(n=100)

+-----------------+-------------+-----+
|Região Geográfica|    Parâmetro|valor|
+-----------------+-------------+-----+
|          SUDESTE|Turbidez (uT)|    8|
|          SUDESTE|Turbidez (uT)| 0,29|
|          SUDESTE|Turbidez (uT)|    0|
|          SUDESTE|Turbidez (uT)|    0|
|          SUDESTE|Turbidez (uT)|    0|
|          SUDESTE|Turbidez (uT)|    8|
|          SUDESTE|Turbidez (uT)|    8|
|          SUDESTE|Turbidez (uT)| 0,48|
|          SUDESTE|Turbidez (uT)|    4|
|          SUDESTE|Turbidez (uT)|    8|
|          SUDESTE|Turbidez (uT)| 0,87|
|          SUDESTE|Turbidez (uT)|    0|
|          SUDESTE|Turbidez (uT)|    3|
|          SUDESTE|Turbidez (uT)|    1|
|          SUDESTE|Turbidez (uT)|    8|
|          SUDESTE|Turbidez (uT)| 0,41|
|          SUDESTE|Turbidez (uT)|  277|
|          SUDESTE|Turbidez (uT)| 0,41|
|          SUDESTE|Turbidez (uT)|    0|
|          SUDESTE|Turbidez (uT)|    3|
|          SUDESTE|Turbidez (uT)|  116|
|          SUDESTE|Turbidez (uT)|  158|


In [46]:
# Limpando a tabela

dfTurbidez.withColumnRenamed('valor','Turbidez').show(n=10)

+-----------------+-------------+--------+
|Região Geográfica|    Parâmetro|Turbidez|
+-----------------+-------------+--------+
|          SUDESTE|Turbidez (uT)|       8|
|          SUDESTE|Turbidez (uT)|    0,29|
|          SUDESTE|Turbidez (uT)|       0|
|          SUDESTE|Turbidez (uT)|       0|
|          SUDESTE|Turbidez (uT)|       0|
|          SUDESTE|Turbidez (uT)|       8|
|          SUDESTE|Turbidez (uT)|       8|
|          SUDESTE|Turbidez (uT)|    0,48|
|          SUDESTE|Turbidez (uT)|       4|
|          SUDESTE|Turbidez (uT)|       8|
+-----------------+-------------+--------+
only showing top 10 rows



In [47]:
dfFlureto = demaisParametros.select('Região Geográfica','Parâmetro','valor').where(demaisParametros['Parâmetro'] == 'Fluoreto (mg/L)')

In [48]:
dfFlureto.withColumnRenamed('valor','Fluoreto').show(n=10)

+-----------------+---------------+--------+
|Região Geográfica|      Parâmetro|Fluoreto|
+-----------------+---------------+--------+
|          SUDESTE|Fluoreto (mg/L)|       8|
|          SUDESTE|Fluoreto (mg/L)|       8|
|          SUDESTE|Fluoreto (mg/L)|    0,71|
|          SUDESTE|Fluoreto (mg/L)|       0|
|          SUDESTE|Fluoreto (mg/L)|       8|
|          SUDESTE|Fluoreto (mg/L)|    0,77|
|          SUDESTE|Fluoreto (mg/L)|       0|
|          SUDESTE|Fluoreto (mg/L)|       8|
|          SUDESTE|Fluoreto (mg/L)|     288|
|          SUDESTE|Fluoreto (mg/L)|     288|
+-----------------+---------------+--------+
only showing top 10 rows



In [49]:
dfColiformes = demaisParametros.select('Região Geográfica','Parâmetro','valor').where(demaisParametros['Parâmetro'] == 'Coliformes totais')

In [50]:
dfColiformes.withColumnRenamed('valor','Coliforme').show(n=10)

+-----------------+-----------------+---------+
|Região Geográfica|        Parâmetro|Coliforme|
+-----------------+-----------------+---------+
|          SUDESTE|Coliformes totais|        8|
|          SUDESTE|Coliformes totais|        0|
|          SUDESTE|Coliformes totais|        8|
|          SUDESTE|Coliformes totais|        8|
|          SUDESTE|Coliformes totais|        0|
|          SUDESTE|Coliformes totais|        8|
|          SUDESTE|Coliformes totais|        8|
|          SUDESTE|Coliformes totais|        0|
|          SUDESTE|Coliformes totais|        8|
|          SUDESTE|Coliformes totais|       10|
+-----------------+-----------------+---------+
only showing top 10 rows



In [51]:
dfEscherichia = demaisParametros.select('Região Geográfica','Parâmetro','valor').where(demaisParametros['Parâmetro'] == 'Escherichia coli')

In [52]:
dfEscherichia.withColumnRenamed('valor','Escheriachia').show(n=10)

+-----------------+----------------+------------+
|Região Geográfica|       Parâmetro|Escheriachia|
+-----------------+----------------+------------+
|          SUDESTE|Escherichia coli|           8|
|          SUDESTE|Escherichia coli|           8|
|          SUDESTE|Escherichia coli|           0|
|          SUDESTE|Escherichia coli|           8|
|          SUDESTE|Escherichia coli|           0|
|          SUDESTE|Escherichia coli|           8|
|          SUDESTE|Escherichia coli|           8|
|          SUDESTE|Escherichia coli|           0|
|          SUDESTE|Escherichia coli|           8|
|          SUDESTE|Escherichia coli|          10|
+-----------------+----------------+------------+
only showing top 10 rows



In [53]:
#Juntando
dfUm = dfEscherichia.join(dfColiformes,dfEscherichia["Região Geográfica"] == dfColiformes["Região Geográfica"])
#Removes duplicate columns
dfUmtratada = dfEscherichia.join(dfColiformes,["Região Geográfica"])


dfDois = dfTurbidez.join(dfFlureto,dfTurbidez["Região Geográfica"] == dfFlureto["Região Geográfica"])
dfDoistratada = dfTurbidez.join(dfFlureto,["Região Geográfica"])

#empDF.join(addDF,empDF["emp_id"] == addDF["emp_id"]).show()
#empDF.join(addDF,["emp_id"]).show()

In [54]:
dfUm.show(n=10)

+-----------------+----------------+-----+-----------------+-----------------+-----+
|Região Geográfica|       Parâmetro|valor|Região Geográfica|        Parâmetro|valor|
+-----------------+----------------+-----+-----------------+-----------------+-----+
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|     CENTRO-OESTE|Colif

In [55]:
dfUmtratada.show(n=1000)

+-----------------+----------------+-----+-----------------+-----+
|Região Geográfica|       Parâmetro|valor|        Parâmetro|valor|
+-----------------+----------------+-----+-----------------+-----+
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    8|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|    0|
|     CENTRO-OESTE|Escherichia coli|    8|Coliformes totais|  

In [56]:
dfDois.show(n=10)

+-----------------+-------------+-----+-----------------+---------------+-----+
|Região Geográfica|    Parâmetro|valor|Região Geográfica|      Parâmetro|valor|
+-----------------+-------------+-----+-----------------+---------------+-----+
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|    8|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)| 2,50|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|    2|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|    6|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|    4|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|    4|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|    0|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)| 27,0|
|     CENTRO-OESTE|Turbidez (uT)|    8|     CENTRO-OESTE|Fluoreto (mg/L)|   12|
|     CENTRO-OESTE|Turbidez (uT)|    8| 

In [57]:
dfDoistratada.show(n=1000)

+-----------------+-------------+-----+---------------+-----+
|Região Geográfica|    Parâmetro|valor|      Parâmetro|valor|
+-----------------+-------------+-----+---------------+-----+
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    8|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)| 2,50|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    2|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    6|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    4|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    4|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    0|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)| 27,0|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|   12|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)| 0,70|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    0|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|   12|
|     CENTRO-OESTE|Turbidez (uT)|    8|Fluoreto (mg/L)|    0|
|     CE

In [58]:
print("Actual columns: ", dfDoistratada.columns)

dataframe = dfDoistratada.withColumnRenamed(
  "Região Geográfica", "regiao").withColumnRenamed(
    "Parâmetro", "p1").withColumnRenamed(
    "valor", "Turbidez").withColumnRenamed(
    "Parâmetro", "p2").withColumnRenamed(
    "valor", "Fluoreto")
  
print("modified columns: ", dataframe.columns)

dataframe.show()


Actual columns:  ['Região Geográfica', 'Parâmetro', 'valor', 'Parâmetro', 'valor']
modified columns:  ['regiao', 'p1', 'Turbidez', 'p1', 'Turbidez']


+------------+-------------+--------+---------------+--------+
|      regiao|           p1|Turbidez|             p1|Turbidez|
+------------+-------------+--------+---------------+--------+
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       8|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|    2,50|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       2|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       6|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       4|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       4|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       0|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|    27,0|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|      12|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|    0,70|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|       0|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|      12|
|CENTRO-OESTE|Turbidez (uT)|       8|Fluoreto (mg/L)|  

In [59]:
print("Actual columns: ", dfUmtratada.columns)

dataframe2 = dfUmtratada.withColumnRenamed(
  "Região Geográfica", "regiao").withColumnRenamed(
    "Parâmetro", "p1").withColumnRenamed(
    "valor", "Escherichia").withColumnRenamed(
    "Parâmetro", "p2").withColumnRenamed(
    "valor", "Coliformes")
  
print("modified columns: ", dataframe2.columns)

dataframe2.show()

Actual columns:  ['Região Geográfica', 'Parâmetro', 'valor', 'Parâmetro', 'valor']
modified columns:  ['regiao', 'p1', 'Escherichia', 'p1', 'Escherichia']


+------------+----------------+-----------+-----------------+-----------+
|      regiao|              p1|Escherichia|               p1|Escherichia|
+------------+----------------+-----------+-----------------+-----------+
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          0|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          0|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          0|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|          8|Coliformes totais|          8|
|CENTRO-OESTE|Escherichia coli|       

In [3]:
df1 = dataframe.drop('p1').show(n=1000)
df2 = dataframe2.drop('p1').show(n=1000)


NameError: name 'dataframe' is not defined

In [71]:
df1trad = df1.selectExpr(" regiao as REGIAO","Turbidez as TURBIDEZ","Turbidez  as FLUERETO")
df2trad = df2.selectExpr(" regiao as REGIAO","Escherichia as ESCHERIA","Escherichia as COLIFORMES")

In [70]:
#unionDF = df1.union(df2)
#unionDF.show(truncate=False)

In [88]:
dfTratada = Dataframe.join(Dataframe2,Dataframe["regiao"] == Dataframe2["regiao"])
dfTratada = Dataframe.join(Dataframe2,["Região Geográfica"])

In [ ]:
# UPANDO OS DADOS TRATADOS COM O 2º TRATAMENTO PARA O CLIENTE UTILIZAR

#dfTratada.write.mode('overwrite').csv('s3a://catarina-3sprint-sptech-bucket-cliente/demais-parametros')